In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [22]:
accidents_data = pd.read_excel('/content/drive/MyDrive/Аварии - датасет.xlsx', parse_dates=['date'])

# Данные о датасете

В наборе все ДТП по Москве за 2020, которые есть в базе ГИБДД. Важно учитывать, что мелкие ДТП, оформленные по европротоколу, в эту базу не попадают (об этом не сказано на сайте, но судя по виду данных и опросу знакомых, попадавших в ДТП за последние годы, тут только ДТП, на которые приехали сотрудники ГИБДД). Поэтому есть определённое смещение в сторону более тяжёлых аварий

* coordL	- координаты места ДТП (долгота)
* coordW	- координаты места ДТП (широта)
* DTPType	- тип ДТП
* date	- дата ДТП
* street	- улица, где произошло ДТП (подтянуто через API Google Maps по координатам)
* road	- улица, где произошло ДТП (данные самого ГИБДД, в основном пропуски)
* NVehicles	- число автомобилей-участников
* NPatricipants	- число людей-участников
* NInjured	- число пострадавших
* NFatal		- число погибших
* additionalFactor	- дополнительные факторы
* lightConditions		- условия освещения
* weatherConditions	- условия погоды
* placeConditions		- условия места ДТП
* closeObjects		- объекты рядом с местом ДТП
* violations		- нарушения, совершённые участниками (через ; для разных участников)
* violationsAdditional	- допонительные нарушения
* alco			- наличия алкоголя в крови участников (через ; для разных участников. Просто набор ";;;" означает отсутствие алкоголя)
* privod			- тип привода автомобилей-участников
* traffic_regime		- изменение режима движения в результате ДТП
* pokritie		- состояние дорожного покрытия
* neispravnosti		- наличие технических неисправностей автомобилей-участников
* safety_belt		- были ли пристёгнуты участники ДТП


In [23]:
accidents_data.shape

(7964, 23)

In [24]:
accidents_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7964 entries, 0 to 7963
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   coordL                7964 non-null   float64       
 1   coordW                7964 non-null   float64       
 2   DTPType               7964 non-null   object        
 3   date                  7964 non-null   datetime64[ns]
 4   street                5996 non-null   object        
 5   road                  1096 non-null   object        
 6   NVehicles             7964 non-null   int64         
 7   NPatricipants         7964 non-null   int64         
 8   NInjured              7964 non-null   int64         
 9   NFatal                7964 non-null   int64         
 10  additionalFactor      7964 non-null   object        
 11  lightConditions       7964 non-null   object        
 12  weatherConditions     7964 non-null   object        
 13  placeConditions   

In [5]:
accidents_data.head(10)

,coordL,coordW,DTPType,date,street,road,NVehicles,NPatricipants,NInjured,NFatal,additionalFactor,lightConditions,weatherConditions,placeConditions,closeObjects,violations,violationsAdditional,alco,privod,traffic_regime,pokritie,neispravnosti,safety_belt
0,37.464817,55.758116,Наезд на препятствие,01.01.2020,ул Мнёвники Нижн.,NaN,1,1,1,0,Сведения отсутствуют,"В темное время суток, освещение включено",Пасмурно,Перегон (нет объектов на месте ДТП),Остановка общественного транспорта,Несоответствие скорости конкретным условиям дв...,Нет нарушений,NaN,С передним приводом,Режим движения не изменялся,Обработанное противогололедными материалами,Технические неисправности отсутствуют,Нет
1,37.388953,55.811338,Наезд на стоящее ТС,01.01.2020,NaN,Московская кольцевая автомобильная дорога (МКА...,2,2,1,0,"Наличие на проезжей части неисправного ТС, меш...",Светлое время суток,Ясно,Перегон (нет объектов на месте ДТП),Отсутствие в непосредственной близости объекто...,Несоответствие скорости конкретным условиям дв...,Нет нарушений,NaN,С передним приводом;С передним приводом,Движение частично перекрыто,Сухое,Технические неисправности отсутствуют;Техничес...,Нет
2,37.623153,55.866787,Наезд на пешехода,01.01.2020,NaN,NaN,1,2,1,0,Сужение проезжей части припаркованным транспортом,Светлое время суток,Пасмурно,Внутридворовая территория,Административные здания,Несоблюдение очередности проезда,Другие нарушения ПДД водителем,NaN,Иное расположение рулевого управления,Режим движения не изменялся,Сухое,Технические неисправности отсутствуют,Нет
3,37.588413,55.660576,Столкновение,01.01.2020,ул Болотниковская,NaN,2,3,1,0,Сведения отсутствуют,"В темное время суток, освещение включено",Пасмурно,Регулируемый пешеходный переход,Остановка общественного транспорта,Нет нарушений;Несоблюдение очередности проезда,Нет нарушений;Нет нарушений,;,С передним приводом;С передним приводом,Движение частично перекрыто,Мокрое,Технические неисправности отсутствуют;Техничес...,Нет;Нет
4,37.636467,55.805098,Падение пассажира,01.01.2020,пр-кт Мира,NaN,1,2,1,0,"Участок, контролируемый камерой автоматической...","В темное время суток, освещение включено",Ясно,Остановка общественного транспорта,Остановка общественного транспорта,Неподача или неправильная подача сигналов пово...,Нарушение правил перевозки людей;Нет нарушений,;,С передним приводом,Режим движения не изменялся,Сухое,Технические неисправности отсутствуют,Нет;Нет
5,37.841678,55.768830,Столкновение,01.01.2020,ул Сталеваров,NaN,2,3,1,0,Сведения отсутствуют,Светлое время суток,Пасмурно,"Мост, эстакада, путепровод",Крупный торговый объект (являющийся объектом м...,Несоответствие скорости конкретным условиям дв...,Нет нарушений,NaN,С передним приводом;С задним приводом,Движение частично перекрыто,Гололедица,Технические неисправности отсутствуют;Техничес...,Нет
6,37.200061,55.999713,Наезд на пешехода,01.01.2020,ул Лётчика Полагушина,NaN,1,2,1,0,"Участок, оборудованный искусственными неровнос...","В темное время суток, освещение включено",Пасмурно,Нерегулируемый пешеходный переход,Крупный торговый объект (являющийся объектом м...,Нарушение правил проезда пешеходного перехода,Оставление места ДТП,NaN,NaN,Режим движения не изменялся,Мокрое,Технические неисправности отсутствуют,Нет
7,37.566183,55.670833,Наезд на стоящее ТС,01.01.2020,NaN,NaN,3,2,1,0,Сужение проезжей части припаркованным транспортом,Светлое время суток,Пасмурно,Внутридворовая территория,Многоквартирные жилые дома,NaN,NaN,NaN,С передним приводом;С передним приводом;С пере...,Движение частично перекрыто,Гололедица,Технические неисправности отсутствуют;Техничес...,NaN
8,37.532365,55.606172,Наезд на препятствие,01.01.2020,ул Тарусская,NaN,1,1,1,0,Сведения отсутствуют,Светлое время суток,Пасмурно,Регулируемый перекресток,Подземный пешеходный переход,Несоответствие скорости конкретным условиям дв...,Нет нарушений,NaN,Полноприводные,Режим движения не изменялся,Мокрое,Технические неисправности отсутствуют,Нет
9,37.276204,55.679726,Наезд на пешехода,01.01.2020,NaN,NaN,1,2,1,0,Свед

In [25]:
accidents_data['viol_list'] = [i.split(';') if isinstance(i, str) else i for i in accidents_data.violations ]

In [26]:
accidents_data['place'] = [accidents_data['street'].iloc[i] if not accidents_data['street'].isna() .iloc[i] else accidents_data['road'].iloc[i] for i in range(accidents_data.shape[0])]

In [27]:
accidents_data.drop(columns=['street', 'road'], inplace=True)

In [4]:
accidents_data['street'].iloc[0]

'ул Мнёвники Нижн.'

In [9]:
accidents_data.groupby('violations').count().index

Index(['Выезд на полосу встречного движения',
       'Выезд на полосу встречного движения в местах, где это запрещено',
       'Выезд на полосу встречного движения с разворотом, поворотом налево или объездом препятствия',
       'Выезд на полосу встречного движения;Нет нарушений',
       'Выезд на полосу встречного движения;Нет нарушений;Нет нарушений',
       'Выезд на полосу встречного движения;Нет нарушений;Нет нарушений;Нет нарушений;Нет нарушений',
       'Движение во встречном направлении по дороге с односторонним движением',
       'Другие нарушения ПДД водителем',
       'Другие нарушения ПДД водителем;Нет нарушений',
       'Другие нарушения ПДД водителями (не применяется с 12.2020)',
       ...
       'Нет нарушений;Нет нарушений;Нет нарушений;Нет нарушений;Выезд на полосу встречного движения;Нет нарушений',
       'Нет нарушений;Нет нарушений;Нет нарушений;Нет нарушений;Нарушение требований сигналов светофора',
       'Нет нарушений;Оставление движущегося транспортного средс

In [29]:
accidents_data['date'].value_counts()

2020-01-20    55
2020-08-31    53
2020-01-21    46
2020-08-12    43
2020-09-17    41
              ..
2020-05-05     5
2020-06-05     4
2020-04-25     4
2020-11-04     3
2020-04-05     2
Name: date, Length: 366, dtype: int64

In [ ]:
accidents_data['lightConditions'].value_counts()

Светлое время суток                            4624
В темное время суток, освещение включено       3193
Сумерки                                          96
В темное время суток, освещение отсутствует      36
В темное время суток, освещение не включено      15
Name: lightConditions, dtype: int64

In [ ]:
accidents_data['violations'].nunique()

127

In [37]:
type(accidents_data.loc[0, 'viol_list'])

list

# Всего нарушений по типам

In [48]:
all_violantions = []
for i in accidents_data['viol_list']:
  if isinstance(i, list):
     all_violantions.extend(i)
count_and_viols = {i: all_violantions.count(i) for i in set(all_violantions)}
all_violantions = pd.Series(count_and_viols.values(), name='Нарушение и их общее количество', index=count_and_viols.keys())
all_violantions.sort_values(ascending=False)


Нет нарушений                                                                                                                                                                                                                                      3750
Неправильный выбор дистанции                                                                                                                                                                                                                       1137
Нарушение правил проезда пешеходного перехода                                                                                                                                                                                                       970
Несоблюдение очередности проезда                                                                                                                                                                                                                    962
Несоотве